# Agents for Amazon Bedrock Runtime

# MISSING:
- EXAMPLE FOR FLOW USE
- EXAMPLES FOR RETRIEVE AND QUERY RELATED METHODS

### Summary

The following actions are supported by Agents for Amazon Bedrock Runtime:

- [ ] DeleteAgentMemory
- [ ] GenerateQuery
- [ ] GetAgentMemory
- [x] InvokeAgent
- [ ] InvokeFlow
- [x] InvokeInlineAgent
- [ ] OptimizePrompt
- [ ] Rerank
- [X] Retrieve
- [ ] RetrieveAndGenerate
- [ ] RetrieveAndGenerateStream

#### Pre-requisites for knowledge base with structured data store

The following two pre-requisites are required for an agent to access a knowledge base with structured data store. Both steps relate to the agent service role, usually created automatically when the agent is created in the console.

Documentation: [Prerequisites for creating an Amazon Bedrock knowledge base with a structured data store
](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-prereq-structured.html)

Documentation Sections Required:
- Attach permissions to your user role:
    - Allow a role to query a knowledge base connected to a structured data store
- Grant database access to the role you use for authentication

Note: To test the knowledge base chat in the console, only the second step is required, using the knowledge base role. Both steps are required for the agent chat in the console or in the python sdk.



#### Utility Functions

In [1]:
import json

def decode_response(response):
    if "completion" not in response:
        raise Exception('Completion not found in response')
    
    events = []
    for event in response["completion"]:
        if 'chunk' in event:
            chunk = event["chunk"]
            part = chunk["bytes"].decode()
            event['chunk']["bytes"] = part

        events.append(event)

    return events

#### Initialize Client

In [2]:
import boto3

client = boto3.client(service_name="bedrock-agent-runtime", region_name="us-east-1")

# agent_id = 'H8ZINCAXLU' # test-temp
# agent_alias_id = 'QA9EFCM1HG' # agent-test-temp-alias1

agent_id = 'GBWMWST3UX' # test-temp-structured
agent_alias_id = 'RI47ABAL41' # version-1-2

---
### Invoke Inline Agent

In [ ]:
response = client.invoke_inline_agent(
    sessionId='uniqueSessionId',
    inputText="What are the main three entities of the database?",
    endSession=False,
    enableTrace=True,
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="You are a helpful assistant that queries data and provides useful responses.",
    knowledgeBases=[{
        'knowledgeBaseId': "WX8MP7WFBI",
        'description': 'Use this KB to get all the info',
    }],
)

#### Display Response Format

In [5]:
response

{'ResponseMetadata': {'RequestId': 'ace7fc3f-37d7-4ebb-806e-4bb6d27b0767',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 29 Jan 2025 14:56:18 GMT',
   'content-type': 'application/vnd.amazon.eventstream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ace7fc3f-37d7-4ebb-806e-4bb6d27b0767',
   'x-amz-bedrock-agent-session-id': 'uniqueSessionId',
   'x-amzn-bedrock-agent-content-type': 'application/json'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'sessionId': 'uniqueSessionId',
 'completion': <botocore.eventstream.EventStream at 0x77fab7f03800>}

#### Display Trace

In [ ]:
events = decode_response(response)
events

#### Final Response

In [31]:
final_response = events[-1]['chunk']['bytes']
final_response

'Based on the search results, the three main entities of the database appear to be:\n\n1. document\n2. document_entity\n3. entity_statistics'

---
### Invoke Agent

In [5]:
session_id = 'test1'

response = client.invoke_agent(
    agentAliasId=agent_alias_id,
    agentId=agent_id,
    sessionId=session_id,
    enableTrace=False,
    inputText='what are the three main entities of the database?',
    # bedrockModelConfigurations={
    #     'performanceConfig': {
    #         'latency': 'standard'
    #     }
    # },
    # endSession=False,
    # memoryId='string',
    # sourceArn='string',
)

# Display answer'
events = decode_response(response)
final_response = events[-1]['chunk']['bytes']
final_response

### Invoke Agent - Structured

In [4]:
session_id = 'test6'

response = client.invoke_agent(
    agentAliasId=agent_alias_id,
    agentId=agent_id,
    sessionId=session_id,
    enableTrace=True,
    inputText='Report on the available data',
    # bedrockModelConfigurations={
    #     'performanceConfig': {
    #         'latency': 'standard'
    #     }
    # },
    # endSession=False,
    # memoryId='string',
    # sourceArn='string',
)

# Display answer'
events = decode_response(response)
final_response = events[-1]['chunk']['bytes']
final_response

"The available data consists of two records, each containing three columns: col1, col2, and col3. The first two columns (col1 and col2) contain long integer values, while the third column (col3) contains string values. The first record has values 1, 2, and 'a' for col1, col2, and col3 respectively. The second record has values 3, 4, and 'b' for the same columns."

---
### Continue the Conversation

Use the same session id.

In [32]:
response = client.invoke_agent(
    agentAliasId=agent_alias_id,
    agentId=agent_id,
    sessionId=session_id,
    enableTrace=True,
    inputText='Which entity did you choose on your previous answer in the conversation history?',
    # bedrockModelConfigurations={
    #     'performanceConfig': {
    #         'latency': 'standard'
    #     }
    # },
    # endSession=False,
    # memoryId='string',
    # sourceArn='string',
)

events = decode_response(response)
final_response = events[-1]['chunk']['bytes']
final_response

'Sorry, I am unable to assist you with this request.'

---
### Use Conversation History with `Session State`

In [121]:
session_state = {
    'conversationHistory': {
        'messages': [
            {
                'content': [
                    {
                        'text': 'The password is "abc1def2". Memorize it.'
                    },
                ],
                'role': 'user'
            },
            {
                'content': [
                    {
                        'text': 'Ok, I have memorized it. The password is "abc1def2".'
                    },
                ],
                'role': 'assistant'
            },
            # { # Commented since the last message should be passed to 'inputText' parameter below
            #     'content': [
            #         {
            #             'text': 'What is the password I asked you to memorize?'
            #         },
            #     ],
            #     'role': 'user'
            # }
        ]
    },
}

response = client.invoke_agent(
    agentAliasId=agent_alias_id,
    agentId=agent_id,
    sessionId='test1234567891',
    enableTrace=True,
    inputText='What is the password I asked you to memorize? Dont search the knowledge base. Only look at our conversation.',
    sessionState=session_state,
)

events = decode_response(response)
final_response = events[-1]['chunk']['bytes']
final_response

---
### Retrieve from Knowledge Base

In [5]:
import boto3

client = boto3.client(service_name="bedrock-agent-runtime", region_name="us-east-1")

# agent_id = 'H8ZINCAXLU'
# agent_alias_id = 'QA9EFCM1HG' # agent-test-temp-alias1
# knowledge_base_id = 'WX8MP7WFBI'
knowledge_base_id = 'VGYT6EXGXM' # structured data store

response = client.retrieve(
    knowledgeBaseId=knowledge_base_id,
    retrievalQuery={
        'text': 'Report on the data available'
    }
)

import json
print(json.dumps(response['retrievalResults'], indent=2))

[
  {
    "content": {
      "row": [
        {
          "columnName": "col1",
          "columnValue": "1",
          "type": "LONG"
        },
        {
          "columnName": "col2",
          "columnValue": "2",
          "type": "LONG"
        },
        {
          "columnName": "col3",
          "columnValue": "a",
          "type": "STRING"
        }
      ],
      "type": "ROW"
    },
    "location": {
      "sqlLocation": {
        "query": "SELECT * FROM public.\"table-test\";"
      },
      "type": "SQL"
    }
  },
  {
    "content": {
      "row": [
        {
          "columnName": "col1",
          "columnValue": "3",
          "type": "LONG"
        },
        {
          "columnName": "col2",
          "columnValue": "4",
          "type": "LONG"
        },
        {
          "columnName": "col3",
          "columnValue": "b",
          "type": "STRING"
        }
      ],
      "type": "ROW"
    },
    "location": {
      "sqlLocation": {
        "query": "SELECT *

### Generate SQL Query

In [ ]:
response = client.generate_query(
    queryGenerationInput={
        'text': 'how many rows the test table have?',
        'type': 'TEXT'
    },
    transformationConfiguration={
        'mode': 'TEXT_TO_SQL',
        'textToSqlConfiguration': {
            'knowledgeBaseConfiguration': {
                'knowledgeBaseArn': 'arn:aws:iam::654395624723:role/service-role/AmazonBedrockExecutionRoleForKnowledgeBase_c4h5j'
            },
            'type': 'KNOWLEDGE_BASE'
        }
    }
)